# Какие группы возьмём в исследование?

TLDR:

Думаем о том, какой запрос задать поисковику vk.com для того, чтобы в выдаче были самые релевантные группы музыкальных коллективов.

Сохраняем всё в файле group_df.csv

In [1]:
 # coding: utf-8
# наше всё
import numpy as np
import pandas as pd

#настройки pandas, с которыми лучше почти всегда
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

import requests
import time

In [2]:
vk_id=7444432
get_token_str='https://oauth.vk.com/authorize?client_id='+str(vk_id)+'&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.52'

### Получение доступа к API vk.com
* Авторизуемся на vk в браузере
* Идём сюда https://vk.com/apps?act=manage
* Регаем приложение (лично я выбрал Standalone-приложение)
* Идём в настройки приложения
* Записываем его vk_id
* Идём по следующей ссылке get_token_str
* После редиректа в урле видет acces token, по умолчанию живёт сутки
* Наслаждаемся, но не забываем про уровни доступа и лимиты
* Для деталей читаем документацию: https://vk.com/dev/first_guide


In [4]:
# функция делает запрос к API vk.com
# запускает поиск сообществ по тексту запроса
def get_top_search(q, position):
    # общий урл почти всех запросов
    url='https://api.vk.com/method/'
    # метод поиска нужных групп
    method='groups.search'
    # параметры конкретного метода
    params = (
        # текст заппроса
        ('q', q), 
        ('v', '5.52'),
        ('access_token', access_token),
    )

    response = requests.get(url+method, params=params)
    
    # json сразу приведём в pandas 
    df=pd.io.json.json_normalize(response.json()['response']['items'])
    
    # мы уже оптимизировали запросы, поэтому выводим только top-1 по каждому запросу
    return df[position-1:position]

In [5]:
# список запросов (можно проверить в браузере или самим запросом)
band_q_lst=[
            'мельница группа', 
            'пикник группа',
            'алиса армия',
            'Король и Шут ',
            'Ночные Снайперы',
            'Сурганова',
            'Земфира ',
            'Мумий Тролль (Mumiy Troll)',
            'Би-2',
            'кипелов ария',
            'Пилот',
            'ДДТ',
            'Сплин',
            'Чайф',
            'FLЁUR / ФЛЁР',
            'Цой',
            'Nautilus Pompilius',
            'стас михайлов',
            'корни группа',
            ]

In [23]:
access_token='не_забудь_вставить_сюда_свежий_токен'

In [7]:
# основной цикл по списку групп, собираёт всё в один датафрейм
res=pd.DataFrame()
for q in band_q_lst:
    try:
        buf=get_top_search(q, 1)
    except:
        buf=pd.DataFrame()
    # здесь задержка в секунду решает все наши проблемы
    time.sleep(1)
    res=pd.concat([res, buf])

In [21]:
# визуально посмотрим и запишем
res

,id,is_closed,name,photo_100,photo_200,photo_50,screen_name,type
0,6016960,0,Группа Мельница,https://sun1-86.userapi.com/c851036/v851036942/1def16/CVvljudzHL4.jpg?ava=1,https://sun1-28.userapi.com/c851036/v851036942/1def15/KdisGxyT4gw.jpg?ava=1,https://sun1-23.userapi.com/c851036/v851036942/1def17/XjXdWrEpn0Y.jpg?ava=1,melnitsamusic,group
1,59992118,0,Группа «Пикник»,https://sun1-20.userapi.com/c856132/v856132130/c688f/OLCwKy3GnL4.jpg?ava=1,https://sun1-93.userapi.com/c856132/v856132130/c688e/9FltsObk29Q.jpg?ava=1,https://sun1-98.userapi.com/c856132/v856132130/c6890/FoKxnfYG0mM.jpg?ava=1,piknik_info,page
3,7373,0,Король и Шут,https://sun1-97.userapi.com/c849228/v849228903/51561/GWjj17ixQzQ.jpg?ava=1,https://sun1-27.userapi.com/c849228/v849228903/51560/vquG8vNO4IE.jpg?ava=1,https://sun1-24.userapi.com/c849228/v849228903/51562/0AZs2JPNhrY.jpg?ava=1,korol_i_shut_ru,group
4,86583346,0,Диана Арбенина. Ночные Снайперы,https://sun1-14.userapi.com/c847221/v847221611/db787/sI3_fmeHMeQ.jpg?ava=1,https://sun1-84.userapi.com/c847221/v847221611/db786/9WY5qR9UO1Y.jpg?ava=1,https://sun1-26.userapi.com/c847221/v847221611/db788/QkP4KAYDcw0.jpg?ava=1,thenightsnipers,group
5,28057628,0,Сурганова и Оркестр. Светлана Сурганова,https://sun1-29.userapi.com/c851436/v851436406/1e1437/pfDEbVJ3I6k.jpg?ava=1,https://sun1-19.userapi.com/c851436/v851436406/1e1436/L9QTLgLkqdk.jpg?ava=1,https://sun1-94.userapi.com/c851436/v851436406/1e1438/g8RIhXq54us.jpg?ava=1,surganovaofficial,page
6,23408047,0,Земфира,https://sun1-47.userapi.com/c206528/v206528778/c65e8/DFpdL4hxc3M.jpg?ava=1,https://sun1-22.userapi.com/c206528/v206528778/c65e7/rWM5QUPcF9w.jpg?ava=1,https://sun1-98.userapi.com/c206528/v206528778/c65e9/5CpGj47pSWw.jpg?ava=1,zemfira,page
7,173891,0,Мумий Тролль (Mumiy Troll),https://sun1-89.userapi.com/NDMc77B36bA6m0z5PDrkFy72SNK8mqIzdxZisg/4pOI6P_8A8o.jpg?ava=1,https://sun1-27.userapi.com/J4IeMh5g-ep4YeB5_Nj0b164i0JZlP1-bX9kow/lE22W64hwlU.jpg?ava=1,https://sun1-47.userapi.com/PehfZBv9bmEhNeEcJNYK_7GtEVwau_0a40CrOw/NukXnNhTvcA.jpg?ava=1,mumiytrollcom,group
8,225666,0,Би-2,https://sun1-15.userapi.com/c857028/v857028429/181f1f/iSe1RpLf_Nk.jpg?ava=1,https://sun1-47.userapi.com/c857028/v857028429/181f1e/PU86JHXeLEQ.jpg?ava=1,https://sun1-15.userapi.com/c857028/v857028429/181f20/fnWnxX122ow.jpg?ava=1,b2band,page
9,4034,0,Любители групп «Ария» и «Кипелов»,https://sun1-84.userapi.com/EO1es7vO_yaWFwDxoO11xtC8SMpEeRd6wEkKVA/UxnNnZ-rNI4.jpg?ava=1,https://sun1-24.userapi.com/ObesgoE6DY7xmu2ubDnnn9nfIMEdTBCIKrcmJg/7H-kDHzDuhA.jpg?ava=1,https://sun1-17.userapi.com/PbvCKgO_-Ajd_wJ2llcLUID_TS6SFBs2kWfS0w/t881yLcvHUs.jpg?ava=1,club4034,group
10,38465,0,Группа ПилОт,https://sun1-28.userapi.com/c857628/v857628917/17f4c6/Uvv_vsDR0Pk.jpg?ava=1,https://sun1-30.userapi.com/c857628/v857628917/17f4c5/aw2YhAVwQZc.jpg?ava=1,https://sun1-14.userapi.com/c857628/v857628917/17f4c7/qcKAiFqPdWc.jpg?ava=1,pilot_spb_ru,group


In [22]:
res.to_csv('group_df.csv', index=False)